Loading dataset

In [2]:
import re
import os


class Dataset:
    def __init__(self):
        self.path = '.././dataset/europarl'

    def load(self):
        texts = []
        labels = []
        for label in os.listdir(self.path):
            if self.ignore_if_undesired_path(label):
                continue
            for file_name in os.listdir(os.path.join(self.path, label)):
                if self.ignore_if_undesired_path(file_name):
                    continue
                labels.append(label)
                with open(os.path.join(self.path, label, file_name), 'rb') as fp:
                    texts.append(self.remove_text_fragment(fp))
        return texts, labels
    
    def ignore_if_undesired_path(self, file: str) -> bool:
        undesired_files = ['.DS_Store']
        if file in undesired_files:
            return True
        return False
    
    def remove_text_fragment(self, fp) -> str:
        r = re.compile('<[^<]+?>')
        return r.sub('', fp.read().decode('utf-8', errors='ignore'))
    



In [3]:
dataset = Dataset()
x, y = dataset.load()
print(f'Length of x: {len(x)} - Sample: {x[0]}')
print(f'Length of y: {len(y)} - Sample: {y[0]}')

Length of x: 28539 - Sample: 
6. Financiamento inovador a nível mundial e europeu (

(FR) Senhora Presidente, lamento que exista uma certa confusão neste relatório da senhora deputada Anni Podimata, que levanta uma série de questões importantes no que diz respeito ao financiamento inovador. Gostaria de pedir ao Grupo da Aliança Progressista dos Socialistas & Democratas no Parlamento Europeu que retirasse a proposta alternativa que apresentou a fim de que o trabalho de redacção conjunto que foi realizado seja concluído.

Em segundo lugar, lamento que uma oposição algo artificial tenha emergido sobre a questão do imposto sobre transacções financeiras e que existam duas alterações muito semelhantes. Lamento que, como resultado disso, o Parlamento não possa expressar o seu apoio esmagador a esse imposto em condições que sejam razoáveis face à competitividade da Europa.

(DE) Senhora Presidente, posso dizer ao senhor deputado Jean-Paul Gauzès o seguinte: sim, tencionamos retirar esta propos

Vectorization

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


cv = CountVectorizer()
X = cv.fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = MultinomialNB()
model.fit(X_train, y_train)
print(f'Model-Score: {model.score(X_test, y_test)}')

Model-Score: 0.9963898916967509


Dump model and vectorizer to disk

In [5]:
import pickle

from typing import Any


class PickleFile:
    @staticmethod
    def load_from_disk(path: str):
        with open(path, 'rb') as f:
            return pickle.load(f)
        
    @staticmethod
    def save_on_disk(content: Any, path: str):
        with open(path, 'wb') as f:
            pickle.dump(content, f)


In [6]:
PickleFile.save_on_disk(
    content = model,
    path = '../model/model.pkl' 
)

PickleFile.save_on_disk(
    content = cv,
    path = '../model/vectorizer.pkl'
)
